<a href="https://colab.research.google.com/github/sandhyaananthan121/OpenCV_Features/blob/main/Feature_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sandhya Ananthan UFID: 1095-3582
# Programming Assignment -2 CAP 5416
# Feature 4 Implementation

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim

# Define hyperparameters for the training and optimization:
n_epochs = 10
batch_size_train = 64
batch_size_test = 1024
learning_rate = 0.01
momentum = 0.5
log_interval = 10

# We will set a fixed random seed to ensure reproducibility of training.
torch.manual_seed(1)

def generate_training_transforms():
  """
  Returns a transformation object which 1) converts images to tensors and 2) normalizes image pixel statistics.
  """
  return transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)) ])

def generate_testing_transforms():
  """
  Returns a transformation object which 1) converts images to tensors and 2) normalizes image pixel statistics.
  """
  return transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)) ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=generate_training_transforms())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=generate_testing_transforms())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size_test,
                                         shuffle=False, num_workers=2)

example = next(iter(trainloader))
type(example)

print("Example is of length {}".format(len(example)))
print("Type of first argument is {}".format(type(example[0])))
print("Type of second argument is {}".format(type(example[1])))

print("First argument has shape: {}".format(example[0].shape))
print("Second argument has shape {}".format(example[1].shape))

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 10)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x

network = Net() # Initialize our model defined above.
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum) # Initialize our optimizer

def train_epoch():
    network.train()  # Put network into "training" mode
    # Assign Extra Variables
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for data, target in trainloader:
        optimizer.zero_grad()  # Make sure all parameters have an initial gradient of 0
        output = network(data)  # Forward pass through the network
        loss = F.cross_entropy(output, target)  # Compute the loss
        loss.backward()  # Backward pass assigns a gradient to every parameter
        optimizer.step()  # The optimizer applies the weight update equation

        # Here, We use the new variable to keep track of average loss and accuracy, first we get total_loss using the below function
        total_loss += loss.item() * data.size(0)

        # As Asked in feature, I used softmax for probablities and dim is the dimention, F.softmax is a torch function.
        probabilities = F.softmax(output, dim=1)
        # Same, told to use argmax for predicted_class. torch.argmax is a torch class function, dim is the dimentions.
        predicted_classes = torch.argmax(probabilities, dim=1)

        # Calculate accuracy
        correct_predictions += (predicted_classes == target).sum().item()
        total_samples += data.size(0)

    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy

# Till Here, the code is kept as it is, we add new test_epoch, where we use torch.no_grad() function for
def test_epoch():
    network.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    # This is a new with function we add no_grad from [2] in a while loop, this makes sure this dataset is not used for training.
    with torch.no_grad():
      # The rest remains the same.
        for data, target in testloader:
            output = network(data)  # Forward pass through the network
            loss = F.cross_entropy(output, target)  # Compute the loss

            # Here, We use the new variable to keep track of average loss and accuracy, first we get total_loss using the below function
            total_loss += loss.item() * data.size(0)

            # As Asked in feature, I used softmax for probablities and dim is the dimention, F.softmax is a torch function.
            probabilities = F.softmax(output, dim=1)
            # Same, told to use argmax for predicted_class. torch.argmax is a torch class function, dim is the dimentions.
            predicted_classes = torch.argmax(probabilities, dim=1)

            # Calculate accuracy
            correct_predictions += (predicted_classes == target).sum().item()
            total_samples += data.size(0)

    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy

"""
for epoch in range(n_epochs):
  # Prints Feature 1 output
    avg_loss, accuracy = train_epoch()
    print(f"Epoch {epoch + 1} of {n_epochs}, Train - Avg Loss: {avg_loss:.4f}, Accuracy: {accuracy * 100:.2f}%")
  # Print the test values for comparision - Feature 2 needed
    test_avg_loss, test_accuracy = test_epoch()
    print(f"Epoch {epoch + 1} of {n_epochs}, Test - Avg Loss: {test_avg_loss:.4f}, Accuracy: {test_accuracy * 100:.2f}%")
"""
# I have kept the code till now just the same above
# Now, I have addded the MLP Class just as requested by the Feature. nn.Module used from torch library given in [3]
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # 4- Blocks as requested, Their linear layers are generated here, the dimentions are previously discussed.
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)  # Last layer outputting logits (no activation)

    def forward(self, x):
        #  This is the relu layer asked in the code Passes the flattened input through the first linear layer (fc1)
        # and applies the Rectified Linear Unit (ReLU) activation function to the output.
        x = torch.flatten(x, 1) # Match dimention
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)  # Output layer (logits, no activation) here, as given no relu used.
        return x

# Since we are using MLP, we need new network and optimizer.


mlp_network = MLP()  # Initialize the MLP model
optimizer = optim.SGD(mlp_network.parameters(), lr=learning_rate, momentum=momentum)  # Initialize the optimizer

# Function to train the MLP for one epoch
def train_epoch_mlp(model, train_loader, optimizer):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for data, target in train_loader:
        optimizer.zero_grad()  # Zero gradients
        output = model(data)  # Forward pass
        loss = F.cross_entropy(output, target)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        # Here, We use the new variable to keep track of average loss and accuracy, first we get total_loss using the below function
        total_loss += loss.item() * data.size(0)

        # As Asked in feature, I used softmax for probablities and dim is the dimention, F.softmax is a torch function.
        probabilities = F.softmax(output, dim=1)
        # Same, told to use argmax for predicted_class. torch.argmax is a torch class function, dim is the dimentions.
        predicted_classes = torch.argmax(probabilities, dim=1)

        # Calculate accuracy
        correct_predictions += (predicted_classes == target).sum().item()
        total_samples += data.size(0)

    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy

# Function to evaluate the MLP on the test set
def test_epoch_mlp(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)  # Forward pass
            loss = F.cross_entropy(output, target)  # Compute the loss

            # Here, We use the new variable to keep track of average loss and accuracy, first we get total_loss using the below function
            total_loss += loss.item() * data.size(0)

            # As Asked in feature, I used softmax for probablities and dim is the dimention, F.softmax is a torch function.
            probabilities = F.softmax(output, dim=1)
            # Same, told to use argmax for predicted_class. torch.argmax is a torch class function, dim is the dimentions.
            predicted_classes = torch.argmax(probabilities, dim=1)

            # Calculate accuracy
            correct_predictions += (predicted_classes == target).sum().item()
            total_samples += data.size(0)

    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy
"""
# Training loop for the MLP
for epoch in range(n_epochs):
    train_loss, train_accuracy = train_epoch_mlp(mlp_network, trainloader, optimizer)
    print(f"Epoch {epoch + 1}/{n_epochs}, Train - Avg Loss: {train_loss:.4f}, Accuracy: {train_accuracy * 100:.2f}%")

    test_loss, test_accuracy = test_epoch_mlp(mlp_network, testloader)
    print(f"Epoch {epoch + 1}/{n_epochs}, Test - Avg Loss: {test_loss:.4f}, Accuracy: {test_accuracy * 100:.2f}%")
"""

# Cnn model asked in Feature 4

class SimpleCNN(nn.Module):
    def __init__(self): # Here since its the CNN layer code, we have  to use convolution,
    # we are supposed to take kernel as 3. The dimentions i have kept the same as MLP.
        super(SimpleCNN, self).__init__() # 4- blocks as asked
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2) # The gives the final pool value
        self.fc = nn.Linear(64 * 4 * 4, 10) # Given block size is 64.

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x))) # Add this line to verify the shape before flattening
        x = x.view(-1, 64 * 4 * 4) # flatening as promised with block size 64.
        x = self.fc(x)
        return x

# New network and optimizer for CNN
cnn_network = SimpleCNN()
cnn_optimizer = optim.SGD(cnn_network.parameters(), lr=learning_rate, momentum=momentum)

def train_epoch_cnn(model, train_loader, optimizer):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        # Here, We use the new variable to keep track of average loss and accuracy, first we get total_loss using the below function
        total_loss += loss.item() * data.size(0)

        # As Asked in feature, I used softmax for probablities and dim is the dimention, F.softmax is a torch function.
        probabilities = F.softmax(output, dim=1)
        # Same, told to use argmax for predicted_class. torch.argmax is a torch class function, dim is the dimentions.
        predicted_classes = torch.argmax(probabilities, dim=1)

        # Calculate accuracy
        correct_predictions += (predicted_classes == target).sum().item()
        total_samples += data.size(0)

    average_loss = total_loss / len(train_loader.dataset)
    accuracy = correct_predictions / len(train_loader.dataset)
    return average_loss, accuracy

def test_epoch_cnn(model, test_loader):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = F.cross_entropy(output, target)

            # Here, We use the new variable to keep track of average loss and accuracy, first we get total_loss using the below function
            total_loss += loss.item() * data.size(0)

            # As Asked in feature, I used softmax for probablities and dim is the dimention, F.softmax is a torch function.
            probabilities = F.softmax(output, dim=1)
            # Same, told to use argmax for predicted_class. torch.argmax is a torch class function, dim is the dimentions.
            predicted_classes = torch.argmax(probabilities, dim=1)

            # Calculate accuracy
            correct_predictions += (predicted_classes == target).sum().item()
            total_samples += data.size(0)

    average_loss = total_loss / len(test_loader.dataset)
    accuracy = correct_predictions / len(test_loader.dataset)
    return average_loss, accuracy

for epoch in range(n_epochs):
    train_loss, train_accuracy = train_epoch_cnn(cnn_network, trainloader, cnn_optimizer)
    print(f"Epoch {epoch + 1}/{n_epochs}, Train - Avg Loss: {train_loss:.4f}, Accuracy: {train_accuracy * 100:.2f}%")

    test_loss, test_accuracy = test_epoch_cnn(cnn_network, testloader)
    print(f"Epoch {epoch + 1}/{n_epochs}, Test - Avg Loss: {test_loss:.4f}, Accuracy: {test_accuracy * 100:.2f}%")

100%|██████████| 170498071/170498071 [00:02<00:00, 70984045.10it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Example is of length 2
Type of first argument is <class 'torch.Tensor'>
Type of second argument is <class 'torch.Tensor'>
First argument has shape: torch.Size([64, 3, 32, 32])
Second argument has shape torch.Size([64])
Epoch 1/10, Train - Avg Loss: 2.2596, Accuracy: 16.55%
Epoch 1/10, Test - Avg Loss: 2.0505, Accuracy: 24.76%
Epoch 2/10, Train - Avg Loss: 1.9050, Accuracy: 30.62%
Epoch 2/10, Test - Avg Loss: 1.7643, Accuracy: 36.51%
Epoch 3/10, Train - Avg Loss: 1.6285, Accuracy: 40.62%
Epoch 3/10, Test - Avg Loss: 1.5945, Accuracy: 42.11%
Epoch 4/10, Train - Avg Loss: 1.4746, Accuracy: 46.44%
Epoch 4/10, Test - Avg Loss: 1.4331, Accuracy: 48.58%
Epoch 5/10, Train - Avg Loss: 1.3651, Accuracy: 51.01%
Epoch 5/10, Test - Avg Loss: 1.3141, Accuracy: 52.65%
Epoch 6/10, Train - Avg Loss: 1.2755, Accuracy: 54.37%
Epoch 6/10, Test - Avg Loss: 1.3917, Accuracy: 51.06%
Epoch 7/10, Train - Avg Loss: 1.1900, 